## 🦜🔗 LangChain Agent mit einem SQL Toolkit


Zuerst enpacken wir unsere Demo Datenbank mit der wir gleich arbeiten werden.


In [ ]:
! [ ! -f northwind.db ] && bunzip2 -k northwind.db.bz2

#### Wir erstellen das SQL Toolkit und den Agenten


In [ ]:
from langchain_community.agent_toolkits.sql.base import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from helpers import llm

model = llm(temperature=0)
db = SQLDatabase.from_uri("sqlite:///northwind.db")
toolkit = SQLDatabaseToolkit(db=db, llm=model)
context = toolkit.get_context()
tools = toolkit.get_tools()

In [ ]:
from langchain_community.agent_toolkits.sql.prompt import SQL_FUNCTIONS_SUFFIX
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

messages = [
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessage(content=SQL_FUNCTIONS_SUFFIX),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
]

prompt = ChatPromptTemplate.from_messages(messages)
prompt = prompt.partial(**context)

from langchain.agents import create_openai_tools_agent
from langchain.agents.agent import AgentExecutor
@chain
def messages_modifier(messages):
    return [
        SystemMessage(system_message_prompt.format(dialect=toolkit.dialect, top_k=10)),
        messages[0],
        AIMessage(
            "I should look at the tables in the database to see what I can query. Then I should query the schema of the most relevant tables."
        ),
        *messages[1:],
    ]


agent = create_openai_tools_agent(model, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    verbose=True,
)

#### Looos....


In [ ]:
input_1 = {"input": "Where do i find the orders?"}
agent_executor.invoke(input_1)

In [ ]:
input_2 = {"input": "Which Employee has the most orders?"}
agent_executor.invoke(input_2)

#### Schauen wir mal, ob er das hier hinbekommt.


In [ ]:
input_3 = {"input": "Which Customer has had the Order with the highest total cost ever? What was the total value and the Order Id?"}
agent_executor.invoke(input_3)

#### Und noch einmal das batchen (async) demonstrieren.


In [ ]:
result = await agent_executor.abatch([input_1, input_2, input_3])


for index, item in enumerate(result):
    print(item)

## ✅ Aufgabe

Ändert alle Namen, die ihr in der Datenbank findet auf 
"Little Bobbie Tables;"
(https://xkcd.com/327/)

In [ ]:
print(db.get_table_info())

In [ ]:
your_input = {"input": ""}
agent_executor.invoke(your_input)
